### Resizing Image to Fit Screen

In [17]:
import pyautogui
import cv2

image_path = "C:/Users/ericz/Mouse Clicker/Games/TyshauneSophie2.png"
#image_path = "C:/Users/ericz/Mouse Clicker/Games/badSheet.jpg"
image = cv2.imread(image_path)

screen_width, screen_height = pyautogui.size()
print(f"Screen Width: {screen_width}, Screen Height: {screen_height}")

image_height, image_width, color_channels = image.shape
print(f"Original Height: {image_height}, Original Width: {image_width}, Original Channels: {color_channels}\n")

resize_metric = 0.8
while True:
    if (image_height < screen_height and image_width < screen_width):
        break

    image_width = int(image_width * resize_metric)
    image_height = int(image_height * resize_metric)
    print("Resizing image to:  ({}, {})\n".format(image_width, image_height))

    image = cv2.resize(image,(image_width, image_height))

# cv2.imshow("image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Screen Width: 1920, Screen Height: 1200
Original Height: 3362, Original Width: 2918, Original Channels: 3

Resizing image to:  (2334, 2689)

Resizing image to:  (1867, 2151)

Resizing image to:  (1493, 1720)

Resizing image to:  (1194, 1376)

Resizing image to:  (955, 1100)



### Click Corners

In [13]:
global click_list 
click_list =[]

def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(x, ',', y) 
        # clicked_image = cv2.circle(img = image, 
        #                            center = (x,y), 
        #                            radius = 7, 
        #                            color = (0,0,255), 
        #                            thickness = -2)
        click_list.append([x,y])

cv2.imshow("image", image)
clicked_image = image
cv2.setMouseCallback("image", click_event)

cv2.waitKey(0)
cv2.destroyAllWindows()
# NOTICE ME: after you finish click, check the top and botton horizontal lines and make sure they are perfectly included and aligne

32 , 43
261 , 42
257 , 1075
13 , 1074


In [10]:
cv2.imshow("image", clicked_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
# hard coded click_list
# click_list = [[112 , 310], [246 , 310], [250 , 967], [116 , 971]]
# click_list = [[112 , 310], [246 , 310], [250 , 969], [116 , 971]] # changed 967 to 969
click_list = [[32 , 43], [261 , 42], [257 , 1075], [13 , 1074]] # for Andrew Sophie
#click_list = [[90 , 271], [240 , 254], [486 , 903], [278 , 984]] # For badSheet.jpg

### Logistics
* Game number, move number, player, etc.

In [19]:
from torchvision.transforms.functional import autocontrast
import numpy as np

# constants (these will not change)
H_SIZE = 30
page = "0"  

gameNumber = "004" # change this for different games
player = "white" # change for white and black
move = 1 # change to 30 when different row

topLeft = [click_list[0][0], click_list[0][1]] # type list for the upper left x and y points
topRight = [click_list[1][0], click_list[1][1]] # upper right x and y points in a list [x, y]
bottomRight = [click_list[2][0], click_list[2][1]]
bottomLeft = [click_list[3][0], click_list[3][1]]

### Main
* This code will extract cells

In [20]:
# New code
import numpy as np
frame = np.copy(image)

# Perspective transformation
pts1 = np.float32([topLeft, topRight, bottomLeft, bottomRight])
pts2 = np.float32([[0, 0], [240 - 90, 0], [0, 600], [240 - 90, 600]]) # nice rec shape, y dividableby 30
matrix = cv2.getPerspectiveTransform(pts1, pts2)
warpedImage = cv2.warpPerspective(frame, matrix, (240 - 90, 600))

# The below is for badSheet.jpg
# New one: click_list = [[90 , 271], [240 , 254], [486 , 903], [278 , 984]]
# pts1 = np.float32([topLeft, topRight, bottomLeft, bottomRight])
# pts2 = np.float32([[0, 0], [240 - 90, 0], [0, 600], [240 - 90, 600]])
# matrix = cv2.getPerspectiveTransform(pts1, pts2)
# warpedImage = cv2.warpPerspective(frame, matrix, (240 - 90, 600))

In [6]:
# Display purpose, can be removed
cv2.imshow('warpedImage', warpedImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
# New code
import os

xLen = 240 - 90
yLen = 600
yDelta = round(yLen / H_SIZE)
yBuffer = 1 # Leave some buffer room
previousbottomLeftPoints = [0, yLen]
previousbottomRightPoints = [xLen, yLen]
bottomLeftPoints = [0, previousbottomLeftPoints[1] - yDelta - yBuffer]
bottomRightPoints = [xLen, previousbottomRightPoints[1] - yDelta - yBuffer]

for ih in range(H_SIZE):
    # calculate coordinates for bottom left and right
    bottomLeftPointsAdj = [bottomLeftPoints[0], max(0, bottomLeftPoints[1] - yBuffer)]
    previousbottomLeftPointsAdj = [previousbottomLeftPoints[0], previousbottomLeftPoints[1] + yBuffer]
    previousbottomRightPointsAdj = [previousbottomRightPoints[0], previousbottomRightPoints[1] + yBuffer]
    bottomRightPointsAdj = [bottomRightPoints[0], max(0, bottomRightPoints[1] - yBuffer)]
    points = [bottomLeftPointsAdj, previousbottomLeftPointsAdj, previousbottomRightPointsAdj, bottomRightPointsAdj]
    print(points)
    points = np.array(points)
    points = points.astype(int)

    rect = cv2.minAreaRect(points) # draw the minumum rectangle possible with these points
    box = cv2.boxPoints(rect) # get the coordinates
    # returns lowest point and goes clockwise
    box = np.intp(box)

    # show images

    # new_image = cv2.drawContours(warpedImage, [box], 0, (0,0,255),1) # contour the box
    # cv2.imshow("cropped image",new_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    name = gameNumber + "_" + page + "_" + str(move) + "_" +player+ ".png"
    directory = "C:/Users/ericz/Mouse Clicker/test"
    SAVED_PATH = os.path.join(directory, name).replace("\\", "/")
    save_image = warpedImage[bottomLeftPointsAdj[1] : previousbottomLeftPointsAdj[1], 0 : previousbottomRightPointsAdj[0]]
    cv2.imwrite(SAVED_PATH, save_image)
    
    move+=1
    
    previousbottomLeftPoints = bottomLeftPoints
    previousbottomRightPoints = bottomRightPoints
    bottomLeftPoints = [0, previousbottomLeftPoints[1] - yDelta]
    bottomRightPoints = [xLen, previousbottomRightPoints[1] - yDelta]    
    
    if ih == 29:
        print(f"We ended bottom Left: {previousbottomLeftPoints}")
        print(f"Theretical bottom Left: {click_list[3]} \n")

        print(f"We ended bottom Right: {previousbottomRightPoints}")
        print(f"Theretical bottom Right: {click_list[2]}")
        break

[[0, 578], [0, 601], [150, 601], [150, 578]]
[[0, 558], [0, 580], [150, 580], [150, 558]]
[[0, 538], [0, 560], [150, 560], [150, 538]]
[[0, 518], [0, 540], [150, 540], [150, 518]]
[[0, 498], [0, 520], [150, 520], [150, 498]]
[[0, 478], [0, 500], [150, 500], [150, 478]]
[[0, 458], [0, 480], [150, 480], [150, 458]]
[[0, 438], [0, 460], [150, 460], [150, 438]]
[[0, 418], [0, 440], [150, 440], [150, 418]]
[[0, 398], [0, 420], [150, 420], [150, 398]]
[[0, 378], [0, 400], [150, 400], [150, 378]]
[[0, 358], [0, 380], [150, 380], [150, 358]]
[[0, 338], [0, 360], [150, 360], [150, 338]]
[[0, 318], [0, 340], [150, 340], [150, 318]]
[[0, 298], [0, 320], [150, 320], [150, 298]]
[[0, 278], [0, 300], [150, 300], [150, 278]]
[[0, 258], [0, 280], [150, 280], [150, 258]]
[[0, 238], [0, 260], [150, 260], [150, 238]]
[[0, 218], [0, 240], [150, 240], [150, 218]]
[[0, 198], [0, 220], [150, 220], [150, 198]]
[[0, 178], [0, 200], [150, 200], [150, 178]]
[[0, 158], [0, 180], [150, 180], [150, 158]]
[[0, 138],